In [ ]:
# default_exp datasets

# Simulated Datasets
The approach for creating simulated datasets is to initialize a model with specified parameters and experience sequences and then populate a psifr-formatted array with the outcomes of performing `free recall`. 

The `simulate_data` function below presumes each item is just presented once and that a model has already been initialized, and is better for quick baseline characterization of model performance. Datasets with item repetitions during presentation violate this premise; a more unique function is normally necessary for simulating these models in a performant way.

Since model simulation this way has always directly led to visualization in work done so far, a corresponding `trials` array is not produced.

In [70]:
# export
import pandas as pd
from psifr import fr
import numpy as np
from numba import int32
from numba import njit

def simulate_df(model, experiment_count, first_recall_item=None):
    """
    Initialize a model with specified parameters and experience sequences and 
    then populate a psifr-formatted dataframe with the outcomes of performing `free recall`. 
    
    **Required model attributes**:
    - item_count: specifies number of items encoded into memory
    - context: vector representing an internal contextual state
    - experience: adding a new trace to the memory model
    - free_recall: function that freely recalls a given number of items or until recall stops
    """
    
    # encode items
    model.experience(model.items)

    # simulate retrieval for the specified number of times, tracking results in df
    data = []
    for experiment in range(experiment_count):
        data += [[experiment, 0, 'study', i + 1, i] for i in range(model.item_count)]
    for experiment in range(experiment_count):
        if first_recall_item is not None:
            model.force_recall(first_recall_item)
        data += [[experiment, 0, 'recall', i + 1, o] for i, o in enumerate(model.free_recall())]
    data = pd.DataFrame(data, columns=['subject', 'list', 'trial_type', 'position', 'item'])
    merged = fr.merge_free_recall(data)
    
    return merged

simulate_data = simulate_df

In [71]:
# export

@njit(fastmath=True, nogil=True)
def simulate_array(model, experiment_count, first_recall_item=None):
    
    # encode items
    model.experience(model.items)

    # simulate retrieval for the specified number of times, tracking results in array
    trials = np.zeros((experiment_count, len(model.items)), dtype=int32)
    
    for trial_index in range(len(trials)):
        
        recalled = model.free_recall()
        trials[trial_index, :len(recalled)] = recalled + 1
        
    return trials

In [72]:
from compmemlearn.fitting import murdock_objective_function, apply_and_concatenate
from compmemlearn.models import Classic_CMR
from compmemlearn.datasets import prepare_murdock1970_data, simulate_data
from scipy.optimize import differential_evolution
from numba.typed import List, Dict
from numba.core import types
from numba import njit
from psifr import fr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

murd_trials0, murd_events0, murd_length0 = prepare_murdock1970_data('../../data/mo1970.txt')

cmr_free_parameters = (
    'encoding_drift_rate',
    'start_drift_rate',
    'recall_drift_rate',
    'shared_support',
    'item_support',
    'learning_rate',
    'primacy_scale',
    'primacy_decay',
    'stop_probability_scale',
    'stop_probability_growth',
    'choice_sensitivity',
    'delay_drift_rate'
)

cmr_result = np.array(
    [5.84259066e-01, 4.27375824e-03, 7.21454638e-01, 8.17704509e-01,
     1.00000000e+00, 9.88623591e-02, 9.31571732e+00, 7.54040329e+01,
     3.14204629e-02, 3.36598109e-01, 9.99452206e+00, 9.95457387e-01])

fitted_parameters = Dict.empty(
    key_type=types.unicode_type, value_type=types.float64)
for i in range(len(cmr_result)):
    fitted_parameters[cmr_free_parameters[i]] = cmr_result[i]
fitted_parameters['sampling_rule'] = 0
fitted_parameters['mfc_familiarity_scale'] = 0
fitted_parameters['mcf_familiarity_scale'] = 0
fitted_parameters['drift_familiarity_scale'] = 0

model = Classic_CMR(murd_length0, murd_length0, fitted_parameters)

sim_df = simulate_data(model, 1000)

sim_df.head()

,subject,list,item,input,output,study,recall,repeat,intrusion
0,0,0,0,1,NaN,True,False,0,False
1,0,0,1,2,NaN,True,False,0,False
2,0,0,2,3,NaN,True,False,0,False
3,0,0,3,4,NaN,True,False,0,False
4,0,0,4,5,NaN,True,False,0,False


In [73]:
murd_trials0[0]

array([15, 16, 17, 18, 20, 11,  0,  0,  0,  0,  0,  0,  0], dtype=int64)

In [74]:
model = Classic_CMR(murd_length0, murd_length0, fitted_parameters)

In [75]:
print(model.context)

for i in range(10):
    print(simulate_array(model, 1000)[:2])
    print(model.context)
    print()

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[18 20 19 17 16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [20  5 13  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
[0.01536349 0.0110604  0.01362845 0.01679275 0.02069176 0.02549605
 0.03141582 0.03871006 0.04769791 0.0587726  0.07241865 0.08923309
 0.10995156 0.13548054 0.16693693 0.20569699 0.2534565  0.312305
 0.38481718 0.4741655  0.58425907 0.        ]

[[21 21 21 21  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [21 21 21 21 21 21 21  0  0  0  0  0  0  0  0  0  0  0  0  0]]
[0.01858157 0.02299512 0.02751675 0.03289876 0.03929717 0.04689425
 0.05590242 0.0665687  0.07917965 0.09406713 0.11161505 0.13226751
 0.15653945 0.18503107 0.21844919 0.25764043 0.3036456  0.35779221
 0.42185708 0.49836164 0.59112984 0.        ]

[[21 21 21 21 21 21 21  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [21 21 21 21 21  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
[0.02834852 0.0392899  0.04603779 0.05387156 0.06294395 0